# 🎯 Fundamentos del Modelo Cuántico de Energías para el Go
## Verificación con 2 Qubits


1. **Mapeo cuántico**: Cómo representar piedras de Go como estados de qubits
2. **Hamiltoniano cuántico**: Construcción del operador de energía
3. **Simetría**: Por qué necesitamos 3 operadores (I⊗Z, Z⊗X, X⊗Z)
4. **Verificación**: Reproducir la tabla de valores esperados

In [1]:
# Configuración del entorno
import sys
sys.path.append('../src')

# Librerías estándar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# PennyLane para computación cuántica
import pennylane as qml

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Librerías importadas correctamente")
print(f"📦 PennyLane versión: {qml.__version__}")

✅ Librerías importadas correctamente
📦 PennyLane versión: 0.42.3


### 🔬 Sistema de 2 Qubits

El ladrillo fundamental es el caso más simple, la interacción de dos qubits:
- **Qubit 0**: Posición central
- **Qubit 1**: Un vecino

Este es el **bloque fundamental** que se repetirá para vecinos múltiples, vecinos definidos por la distancia Manhattan del **Qubit 0**.

### Representación Cuántica vs Clásica

| Piedra | Clásico (spin) | Cuántico (qubit) | Notación |
|--------|----------------|------------------|----------|
| Blanco | s = +1         | \|0⟩            | W, white |
| Negro  | s = -1         | \|1⟩            | B, black |
| Vacío  | s = 0          | \|+⟩ o \|−⟩     | E, empty |

**Superposiciones para espacios vacíos:**
```
|+⟩ = (|0⟩ + |1⟩)/√2   # Equiprobable, sin fase
|−⟩ = (|0⟩ - |1⟩)/√2   # Equiprobable, con fase π (sobra esta opcion, no la usaremos)
```

## ⚛️ Construcción del Hamiltoniano

La construcción de un Hamiltoniano adecuado es crucial para capturar la dinámica del sistema la cual esta definida por la tabla de energías correspondientes a cada combinación posible de dos qubits. Este Hamiltoniano debe reflejar las interacciones entre el qubit que es el anclaje y su entorno dentro del kernel definido por la distancia Manhattan. Entonces en cierta manera es arbitraria la elección de los operadores a usar.

El Hamiltoniano propuesto tiene 3 términos y expresa las interacciones de dos qubits:
```
H = I⊗Z + Z⊗X + X⊗Z
```

de manera que refleja simetría en la interacción de colores en las piedras de go.

1. **I⊗Z**: Mide el estado intrínseco del vecino
   - No depende del centro
   - Eigenvalores: +1 (blanco), -1 (negro), 0 (vacío)

2. **Z⊗X**: Centro influye sobre vecino
   - Z del centro distingue blanco/negro
   - X del vecino mide su "susceptibilidad" al cambio

3. **X⊗Z**: Vecino influye sobre centro
   - Simétrico al anterior
   - Crucial cuando el centro está vacío

In [2]:
def create_hamiltonian():
    """
    Crea el Hamiltoniano de Ising cuántico para 2 qubits.
    
    Returns:
        qml.Hamiltonian
    """
    # Coeficientes (todos = 1.0 para simplificar)
    coeffs = [1.0, 1.0, 1.0]
    
    # Operadores
    observables = [
        qml.Identity(0) @ qml.PauliZ(1),   # I⊗Z
        qml.PauliZ(0) @ qml.PauliX(1),     # Z⊗X
        qml.PauliX(0) @ qml.PauliZ(1)      # X⊗Z
    ]
    
    return qml.Hamiltonian(coeffs, observables)

def initialize_qubit_from_char(wire, char):
    """
    Inicializa un qubit según un carácter.
    
    Args:
        wire: Índice del qubit (0 o 1)
        char: '0' (blanco), '1' (negro), '+' (vacío+), '-' (vacío-)
    """
    if char == '0':
        # |0⟩ - estado por defecto, no hacer nada
        pass
    elif char == '1':
        # |1⟩ - aplicar X (bit flip)
        qml.PauliX(wires=wire)
    elif char == '+':
        # |+⟩ = H|0⟩
        qml.Hadamard(wires=wire)
    elif char == '-':
        # |−⟩ = H·Z|0⟩
        qml.Hadamard(wires=wire)
        qml.PauliZ(wires=wire)
    else:
        raise ValueError(f"Carácter inválido: {char}")

def initialize_two_qubits(state_str):
    """
    Inicializa ambos qubits a partir de un string.
    
    Args:
        state_str: String de 2 caracteres, ej: "01", "+0", etc.
    """
    initialize_qubit_from_char(0, state_str[0])  # Qubit 0
    initialize_qubit_from_char(1, state_str[1])  # Qubit 1


In [3]:
state_interpretations = {
    "00": "Blanco - Blanco",
    "01": "Blanco - Negro",
    "0+": "Blanco - Vacío (+)",
    "0-": "Blanco - Vacío (-)",
    "10": "Negro - Blanco",
    "11": "Negro - Negro",
    "1+": "Negro - Vacío (+)",
    "1-": "Negro - Vacío (-)",
    "+0": "Vacío (+) - Blanco",
    "+1": "Vacío (+) - Negro",
    "++": "Vacío (+) - Vacío (+)",
    "+-": "Vacío (+) - Vacío (-)",
    "-0": "Vacío (-) - Blanco",
    "-1": "Vacío (-) - Negro",
    "-+": "Vacío (-) - Vacío (+)",
    "--": "Vacío (-) - Vacío (-)"
}

In [ ]:
# Crear Hamiltoniano
H = create_hamiltonian()
# Crear dispositivo cuántico (simulador)
dev = qml.device('default.qubit', wires=2)

@qml.qnode(dev)
def circuit(state_str):
    """
    Circuito cuántico que calcula ⟨ψ|H|ψ⟩.
    
    Args:
        state_str: Estado a preparar (ej: "01")
        
    Returns:
        Valor esperado del Hamiltoniano
    """
    # Preparar estado
    initialize_two_qubits(state_str)
    
    # Medir energía
    return qml.expval(H)

print("⚡ Hamiltoniano construido:\n")
print(H)
print("\n📊 Información:")
print(f"  - Número de términos: {len(H.coeffs)}")
print(f"  - Coeficientes: {H.coeffs}")

######################################################################
test_state = "10"  # Probar con un estado simple que desees
test_energy = circuit(test_state)

print(f"🧪 Prueba del circuito:")
print(f"  Estado: |{test_state}⟩ ({state_interpretations[test_state]})")
print(f"  Energía: {test_energy:.4f}")

# Dibujar el circuito para visualización
print("\n📐 Estructura del circuito:")
print(qml.draw(circuit)(test_state))

⚡ Hamiltoniano construido:

1.0 * (I(0) @ Z(1)) + 1.0 * (Z(0) @ X(1)) + 1.0 * (X(0) @ Z(1))

📊 Información:
  - Número de términos: 3
  - Coeficientes: [1.0, 1.0, 1.0]
🧪 Prueba del circuito:
  Estado: |10⟩ (Negro - Blanco)
  Energía: 1.0000

📐 Estructura del circuito:
0: ──X─┤ ╭<𝓗(1.00,1.00,1.00)>
1: ────┤ ╰<𝓗(1.00,1.00,1.00)>


### 3.1 Análisis de Casos Clave

**Caso 1: |0,0⟩ - Dos blancos**
```
⟨0,0|H|0,0⟩ = ⟨0,0|I⊗Z|0,0⟩ + ⟨0,0|Z⊗X|0,0⟩ + ⟨0,0|X⊗Z|0,0⟩
            = 1 + 0 + 0 = +1.0
```
✓ Energía positiva (configuración estable para blanco)

**Caso 2: |1,1⟩ - Dos negros**
```
⟨1,1|H|1,1⟩ = ⟨1,1|I⊗Z|1,1⟩ + ...
            = -1 + 0 + 0 = -1.0
```
✓ Energía negativa (configuración estable para negro)

**Caso 3: |0,1⟩ - Blanco vecino de negro**
```
⟨0,1|H|0,1⟩ = -1.0
```
✗ Energía negativa (configuración inestable, conflicto)

**Caso 4: |+,0⟩ - Vacío vecino de blanco**
```
⟨+,0|H|+,0⟩ = 2.0
```
✓ **Energía DOBLE positiva** → El vacío tiene fuerte influencia blanca
- Factor 2 porque el vacío es influenciado **sin tener identidad propia**

**Caso 5: |+,+⟩ - Dos vacíos**
```
⟨+,+|H|+,+⟩ = 0.0

In [8]:
# Verificacion: circuito vs valor analitico, usando TU H y TU circuit

import pandas as pd

# 1) Estados desde tu diccionario
states = list(state_interpretations.keys())

# 2) Ejecutar tu circuito tal cual (usa firma circuit(idx, states) o circuit(state_str))
def run_circuit_on_state(s: str) -> float:
    # Soporta ambas firmas de tu cuaderno
    try:
        # version: circuit(state_str)
        return float(circuit(s))
    except TypeError:
        # version: circuit(idx, states)
        idx = states.index(s)
        return float(circuit(idx, states))

# 3) Valor analitico en estados producto para H con PauliI/PauliZ/PauliX
_ex = {
    '0': {'Z':  1.0, 'X':  0.0, 'I': 1.0},
    '1': {'Z': -1.0, 'X':  0.0, 'I': 1.0},
    '+': {'Z':  0.0, 'X':  1.0, 'I': 1.0},
    '-': {'Z':  0.0, 'X': -1.0, 'I': 1.0},
}

def _term_expect_on_state(term, s: str) -> float:
    # term puede ser un observable simple (Z(0)) o un Tensor (Z(0) @ X(1))
    # Extraemos factores por wire
    def _factors(t):
        if hasattr(t, "obs"):       # Tensor.obs
            return list(t.obs)
        if hasattr(t, "operands"):  # Tensor.operands
            return list(t.operands)
        return [t]
    # Inicialmente identidad en ambos qubits
    fac_by_wire = {0: 'I', 1: 'I'}
    for fac in _factors(term):
        name = getattr(fac, "name", str(fac))
        wires = list(getattr(fac, "wires", []))
        if not wires:
            continue
        w = int(wires[0])
        # Normalizar nombres
        if "PauliZ" in name: fac_by_wire[w] = 'Z'
        elif "PauliX" in name: fac_by_wire[w] = 'X'
        elif "Identity" in name: fac_by_wire[w] = 'I'
        else:
            raise ValueError(f"Operador no soportado en valor analitico: {name}")
    # Producto de expectativas por qubit
    return _ex[s[0]][fac_by_wire[0]] * _ex[s[1]][fac_by_wire[1]]

def analytic_energy(H, s: str) -> float:
    total = 0.0
    for c, term in zip(H.coeffs, H.ops):
        total += float(c) * _term_expect_on_state(term, s)
    return total

# 4) Construir tabla comparando circuito vs analitico
rows = []
for s in states:
    e_circ = run_circuit_on_state(s)
    e_exp = analytic_energy(H, s)
    rows.append({
        'Estado': s,
        'Descripcion': state_interpretations.get(s, ''),
        'Energia_circuito': e_circ,
        'Energia_analitica': e_exp,
        'Delta': round(e_circ - e_exp, 8),
    })
df = pd.DataFrame(rows).sort_values('Estado').reset_index(drop=True)
df['OK'] = df['Delta'].apply(lambda d: 'OK' if abs(d) < 1e-8 else 'X')
display(df)

# 5) Resumen
max_abs_delta = float(df['Delta'].abs().max())
num_ok = int((df['Delta'].abs() < 1e-8).sum())
total = len(df)
print("Resumen verificacion")
print(f"- Max |circuito - analitico|: {max_abs_delta:.6e}")
print(f"- Estados correctos: {num_ok}/{total} ({100.0*num_ok/total:.1f}%)")


,Estado,Descripcion,Energia_circuito,Energia_analitica,Delta,OK
0,++,Vacío (+) - Vacío (+),0.0,0.0,0.0,OK
1,+-,Vacío (+) - Vacío (-),0.0,0.0,0.0,OK
2,+0,Vacío (+) - Blanco,2.0,2.0,-0.0,OK
3,+1,Vacío (+) - Negro,-2.0,-2.0,0.0,OK
4,-+,Vacío (-) - Vacío (+),0.0,0.0,0.0,OK
5,--,Vacío (-) - Vacío (-),0.0,0.0,0.0,OK
6,-0,Vacío (-) - Blanco,0.0,0.0,0.0,OK
7,-1,Vacío (-) - Negro,0.0,0.0,0.0,OK
8,0+,Blanco - Vacío (+),1.0,1.0,-0.0,OK
9,0-,Blanco - Vacío (-),-1.0,-1.0,0.0,OK


Resumen verificacion
- Max |circuito - analitico|: 0.000000e+00
- Estados correctos: 16/16 (100.0%)


**Hamiltoniano Clásico de Ising:**
```
H_classical(t) = -Σ s_i^(t) - μ·Σ h_i^(t)·x_i^(t)
```

**Donde:**
- `s_i`: Energía local de interacción
- `h_i`: Número de libertades de la piedra i
- `x_i`: Spin de la piedra i (-1=negro, +1=blanco, 0=vacío)
- `μ`: Parámetro que controla la importancia de las libertades

**Energía Local:**
```
s_i = x_i · Σ(x_j para j vecinos de i)
```

Una piedra tiene energía favorable si está rodeada de piedras del mismo color.

---

### 🔬 Diferencias Clave: Clásico vs Cuántico

| Aspecto | Clásico | Cuántico |
|---------|---------|----------|
| **Estados** | Discretos: {-1, 0, +1} | Superposición: α\|0⟩ + β\|1⟩ |
| **Vacíos** | Spin = 0 (sin energía) | \|+⟩ o \|−⟩ (superposición) |
| **Interacción** | Producto de spins | Operadores tensoriales |
| **Libertades** | Campo magnético | Término I⊗Z |


In [10]:
# Helpers minimos faltantes: classical_params_from_quantum y classical_energy
# No redefinen H ni circuit. Se crean solo si no existen.

if 'classical_params_from_quantum' not in globals():
    def classical_params_from_quantum(H):
        # Extrae coeficientes de (I@Z), (Z@X), (X@Z) desde H.ops/H.coeffs
        def _op_label(fac):
            name = getattr(fac, "name", str(fac))
            if "PauliZ" in name: return 'Z'
            if "PauliX" in name: return 'X'
            if "Identity" in name: return 'I'
            return None

        def _factors(term):
            if hasattr(term, "obs"):       # bokeh>=? / pl>=? tensor
                return list(term.obs)
            if hasattr(term, "operands"):  # versiones previas
                return list(term.operands)
            return [term]

        c_IZ = c_ZX = c_XZ = 0.0
        for c, term in zip(H.coeffs, H.ops):
            labels = {0: 'I', 1: 'I'}
            for fac in _factors(term):
                wires = list(getattr(fac, "wires", []))
                if not wires:
                    continue
                w = int(wires[0])
                lab = _op_label(fac) or 'I'
                labels[w] = lab
            key = (labels[0], labels[1])
            if key == ('I','Z'):
                c_IZ += float(c)
            elif key == ('Z','X'):
                c_ZX += float(c)
            elif key == ('X','Z'):
                c_XZ += float(c)

        # Parametros clasicos para E = h0*s0 + h1*s1 + J*s0*s1 + K*s0*s1^2 + L*s0^2*s1
        h0 = c_ZX
        h1 = c_IZ + c_XZ
        J  = 0.0
        K  = -c_ZX
        L  = -c_XZ
        return dict(h0=h0, h1=h1, J=J, K=K, L=L)

if 'classical_energy' not in globals():
    def _s_value(ch: str) -> float:
        # s in {-1,0,1}: 0->+1, 1->-1, +/- -> 0
        return 1.0 if ch == '0' else (-1.0 if ch == '1' else 0.0)

    def classical_energy(params, s: str) -> float:
        h0, h1, J, K, L = params['h0'], params['h1'], params['J'], params['K'], params['L']
        s0, s1 = _s_value(s[0]), _s_value(s[1])
        return (h0*s0 + h1*s1 + J*s0*s1 + K*s0*(s1**2) + L*(s0**2)*s1)


In [13]:
# CUBICA -> CUADRATICA con variables auxiliares de ocupacion o_i = s_i^2

import itertools
import pandas as pd

# 1) Obtener parametros clasicos (si ya los tienes en 'params', se usan tal cual)
if 'params' in globals():
    h0, h1, J, K, L = params['h0'], params['h1'], params['J'], params['K'], params['L']
else:
    # Si tienes H y classical_params_from_quantum(H), los derivamos
    assert 'H' in globals(), "No hay 'params' ni 'H'. Define alguno antes."
    if 'classical_params_from_quantum' not in globals():
        # Fallback minimo para extraer (I@Z, Z@X, X@Z) de H y derivar h0,h1,J,K,L
        def _op_label(fac):
            name = getattr(fac, "name", str(fac))
            if "PauliZ" in name: return 'Z'
            if "PauliX" in name: return 'X'
            if "Identity" in name: return 'I'
            return None
        def _factors(term):
            if hasattr(term, "obs"): return list(term.obs)
            if hasattr(term, "operands"): return list(term.operands)
            return [term]
        def _extract_coeffs_IZ_ZX_XZ(H):
            c_IZ = c_ZX = c_XZ = 0.0
            for c, term in zip(H.coeffs, H.ops):
                labels = {0:'I', 1:'I'}
                for fac in _factors(term):
                    wires = list(getattr(fac, "wires", []))
                    if not wires: continue
                    w = int(wires[0]); lab = _op_label(fac) or 'I'
                    labels[w] = lab
                key = (labels[0], labels[1])
                if key == ('I','Z'): c_IZ += float(c)
                elif key == ('Z','X'): c_ZX += float(c)
                elif key == ('X','Z'): c_XZ += float(c)
            return c_IZ, c_ZX, c_XZ
        def classical_params_from_quantum(H):
            c_IZ, c_ZX, c_XZ = _extract_coeffs_IZ_ZX_XZ(H)
            # E = h0*s0 + h1*s1 + J*s0*s1 + K*s0*s1^2 + L*s0^2*s1
            return dict(h0=c_ZX, h1=c_IZ+c_XZ, J=0.0, K=-c_ZX, L=-c_XZ)
    p = classical_params_from_quantum(H)
    h0, h1, J, K, L = p['h0'], p['h1'], p['J'], p['K'], p['L']

# 2) Definiciones de energia (cúbica y cuadrática)
def E_cubic(s0, s1):
    return h0*s0 + h1*s1 + J*s0*s1 + K*s0*(s1*s1) + L*(s0*s0)*s1

def E_quad(s0, s1, o0, o1):
    # o0 = s0^2, o1 = s1^2
    return h0*s0 + h1*s1 + J*s0*s1 + K*s0*o1 + L*o0*s1

# 3) Verificacion exhaustiva en s0,s1 ∈ {-1,0,+1} con o_i = s_i^2
vals = [-1, 0, 1]
rows = []
for s0, s1 in itertools.product(vals, vals):
    o0, o1 = s0*s0, s1*s1  # ocupaciones correctas
    ec = E_cubic(s0, s1)
    eq = E_quad(s0, s1, o0, o1)
    rows.append({
        's0': s0, 's1': s1, 'o0': o0, 'o1': o1,
        'E_cubica': round(ec, 6),
        'E_cuadratica(o=s^2)': round(eq, 6),
        'Delta': round(eq - ec, 6)
    })

df = pd.DataFrame(rows).sort_values(['s0','s1']).reset_index(drop=True)
display(df)
print("Max |Delta| (debe ser 0):", df['Delta'].abs().max())

# Nota: Si alguien usara o0, o1 incorrectos (no iguales a s0^2, s1^2),
# esta forma cuadratica NO es equivalente; la equivalencia es exacta cuando o_i = s_i^2.


,s0,s1,o0,o1,E_cubica,E_cuadratica(o=s^2),Delta
0,-1,-1,1,1,-1.0,-1.0,0.0
1,-1,0,1,0,-1.0,-1.0,0.0
2,-1,1,1,1,1.0,1.0,0.0
3,0,-1,0,1,-2.0,-2.0,0.0
4,0,0,0,0,0.0,0.0,0.0
5,0,1,0,1,2.0,2.0,0.0
6,1,-1,1,1,-1.0,-1.0,0.0
7,1,0,1,0,1.0,1.0,0.0
8,1,1,1,1,1.0,1.0,0.0


Max |Delta| (debe ser 0): 0.0


In [11]:
# Resumen mínimo: analitico vs circuito vs clasico (sin redefinir helpers)

import pandas as pd

# Verificaciones suaves para no duplicar trabajo
required = ['analytic_energy', 'classical_params_from_quantum', 'classical_energy', 'H', 'circuit', 'state_interpretations']
missing = [name for name in required if name not in globals()]
assert not missing, f"Faltan en memoria: {missing}. Ejecuta primero el bloque 'Analitico a partir de H'."

states = list(state_interpretations.keys())
params = classical_params_from_quantum(H)

rows = []
for s in states:
    e_a = analytic_energy(H, s)
    # soporta tu firma circuit(state_str); si usas circuit(idx, states), usa esta alternativa:
    try:
        e_c = float(circuit(s))
    except TypeError:
        idx = states.index(s)
        e_c = float(circuit(idx, states))
    e_cls = classical_energy(params, s)
    rows.append({
        'Estado': s,
        'Descripcion': state_interpretations[s],
        'E_analitica': e_a,
        'E_circuito': e_c,
        'E_clasica': e_cls,
        'Delta(analitica-circuito)': round(e_a - e_c, 8),
        'Delta(analitica-clasica)': round(e_a - e_cls, 8),
    })

df = pd.DataFrame(rows).sort_values('Estado').reset_index(drop=True)
display(df)

print("Parametros clasicos (E = h0*s0 + h1*s1 + J*s0*s1 + K*s0*s1^2 + L*s0^2*s1):")
print(classical_params_from_quantum(H))

d1 = df['Delta(analitica-circuito)'].abs().max()
d2 = df['Delta(analitica-clasica)'].abs().max()
print(f"Max |analitica - circuito|: {d1:.3e}")
print(f"Max |analitica - clasica|:  {d2:.3e}")


,Estado,Descripcion,E_analitica,E_circuito,E_clasica,Delta(analitica-circuito),Delta(analitica-clasica)
0,++,Vacío (+) - Vacío (+),0.0,0.0,0.0,0.0,0.0
1,+-,Vacío (+) - Vacío (-),0.0,0.0,0.0,0.0,0.0
2,+0,Vacío (+) - Blanco,2.0,2.0,2.0,0.0,0.0
3,+1,Vacío (+) - Negro,-2.0,-2.0,-2.0,-0.0,0.0
4,-+,Vacío (-) - Vacío (+),0.0,0.0,0.0,0.0,0.0
5,--,Vacío (-) - Vacío (-),0.0,0.0,0.0,0.0,0.0
6,-0,Vacío (-) - Blanco,0.0,0.0,2.0,0.0,-2.0
7,-1,Vacío (-) - Negro,0.0,0.0,-2.0,0.0,2.0
8,0+,Blanco - Vacío (+),1.0,1.0,1.0,0.0,0.0
9,0-,Blanco - Vacío (-),-1.0,-1.0,1.0,-0.0,-2.0


Parametros clasicos (E = h0*s0 + h1*s1 + J*s0*s1 + K*s0*s1^2 + L*s0^2*s1):
{'h0': 1.0, 'h1': 2.0, 'J': 0.0, 'K': -1.0, 'L': -1.0}
Max |analitica - circuito|: 0.000e+00
Max |analitica - clasica|:  2.000e+00


In [12]:
# Quadratic Ising/QUBO a partir de E(s0,s1) = h0 s0 + h1 s1 + K s0 s1^2 + L s0^2 s1
# Codificacion: s_i = p_i - n_i, o_i = p_i + n_i, p_i,n_i ∈ {0,1} y p_i + n_i ≤ 1

import itertools
import pandas as pd

# 1) Parametros clasicos (si no existen, derivarlos desde H)
if 'classical_params_from_quantum' not in globals():
    def _op_label(fac):
        name = getattr(fac, "name", str(fac))
        if "PauliZ" in name: return 'Z'
        if "PauliX" in name: return 'X'
        if "Identity" in name: return 'I'
        return None
    def _factors(term):
        if hasattr(term, "obs"): return list(term.obs)
        if hasattr(term, "operands"): return list(term.operands)
        return [term]
    def _extract_coeffs_IZ_ZX_XZ(H):
        c_IZ = c_ZX = c_XZ = 0.0
        for c, term in zip(H.coeffs, H.ops):
            labels = {0:'I', 1:'I'}
            for fac in _factors(term):
                wires = list(getattr(fac, "wires", []))
                if not wires: continue
                w = int(wires[0]); lab = _op_label(fac) or 'I'
                labels[w] = lab
            key = (labels[0], labels[1])
            if key == ('I','Z'): c_IZ += float(c)
            elif key == ('Z','X'): c_ZX += float(c)
            elif key == ('X','Z'): c_XZ += float(c)
        return c_IZ, c_ZX, c_XZ
    def classical_params_from_quantum(H):
        c_IZ, c_ZX, c_XZ = _extract_coeffs_IZ_ZX_XZ(H)
        # E = h0*s0 + h1*s1 + J*s0*s1 + K*s0*s1^2 + L*s0^2*s1; (aqui J=0)
        return dict(h0=c_ZX, h1=c_IZ+c_XZ, J=0.0, K=-c_ZX, L=-c_XZ)

params = classical_params_from_quantum(H)
h0, h1, J, K, L = params['h0'], params['h1'], params['J'], params['K'], params['L']

# 2) Construir QUBO (lineal y cuadratico) sobre variables {p0,n0,p1,n1}
vars_ = ['p0','n0','p1','n1']
idx = {v:i for i,v in enumerate(vars_)}

# Bias lineales (b) y acoples (Q) en matriz 4x4 (simetrica, upper-tri usada)
import numpy as np
b = np.zeros(4, dtype=float)
Q = np.zeros((4,4), dtype=float)

# E = h0(p0-n0) + h1(p1-n1) + K(p0-n0)(p1+n1) + L(p0+n0)(p1-n1)
# Terminos lineales:
b[idx['p0']] += h0
b[idx['n0']] += -h0
b[idx['p1']] += h1
b[idx['n1']] += -h1

# Terminos cuadraticos (usar upper-tri; sumarlos simetricamente al evaluar)
def add_quad(u, v, coef):
    i, j = idx[u], idx[v]
    if i == j:
        b[i] += coef  # x^2 = x para binaria -> se puede plegar en lineal
    elif i < j:
        Q[i,j] += coef
    else:
        Q[j,i] += coef

# K(p0 p1 + p0 n1 - n0 p1 - n0 n1)
add_quad('p0','p1',  K)
add_quad('p0','n1',  K)
add_quad('n0','p1', -K)
add_quad('n0','n1', -K)

# L(p0 p1 - p0 n1 + n0 p1 - n0 n1)
add_quad('p0','p1',  L)
add_quad('p0','n1', -L)
add_quad('n0','p1',  L)
add_quad('n0','n1', -L)

# 3) Penalizacion para p_i + n_i ≤ 1 -> penalizar p_i n_i
P = 10.0 * (1 + abs(h0)+abs(h1)+abs(K)+abs(L))  # grande para que respete la restriccion
add_quad('p0','n0', P)
add_quad('p1','n1', P)

print("QUBO lineal b:", dict(zip(vars_, b)))
print("QUBO cuadratico Q (upper-tri no nulo):")
for i in range(4):
    for j in range(i+1,4):
        if Q[i,j] != 0.0:
            print(f"  {vars_[i]}*{vars_[j]}: {Q[i,j]}")

# 4) Validacion rapida contra energia clasica/analitica en estados sin '-'
#   Mapeo (p0,n0,p1,n1) -> s0,s1: s_i = p_i - n_i; o_i = p_i + n_i
#   Restringimos a combinaciones validas p_i+n_i in {0,1}
def energy_qubo(p0,n0,p1,n1):
    x = np.array([p0,n0,p1,n1], dtype=float)
    return float(b @ x + x @ Q @ x)  # x^T Q x + b^T x

def s_from(p,n): return p - n
def o_from(p,n): return p + n

# Energia clasica original (cúbica) con s y o = s^2
def E_cubic(s0,s1):
    return h0*s0 + h1*s1 + K*s0*(s1*s1) + L*(s0*s0)*s1

rows = []
for p0,n0,p1,n1 in itertools.product([0,1],[0,1],[0,1],[0,1]):
    if (p0+n0) > 1 or (p1+n1) > 1:
        continue
    s0, s1 = s_from(p0,n0), s_from(p1,n1)
    # Evitar el caso con '-' si no te interesa: aqui no filtramos, pero puedes filtrar s=0/±1 como gustes
    eq = energy_qubo(p0,n0,p1,n1)
    ec = E_cubic(s0,s1)
    rows.append({
        'p0':p0,'n0':n0,'p1':p1,'n1':n1,
        's0':s0,'s1':s1,
        'E_qubo': round(eq,6),
        'E_cubica': round(ec,6),
        'Delta': round(eq-ec,6)
    })

df = pd.DataFrame(rows).sort_values(['s0','s1','p0','n0','p1','n1']).reset_index(drop=True)
display(df)
print("Max |Delta|:", df['Delta'].abs().max())


QUBO lineal b: {'p0': np.float64(1.0), 'n0': np.float64(-1.0), 'p1': np.float64(2.0), 'n1': np.float64(-2.0)}
QUBO cuadratico Q (upper-tri no nulo):
  p0*n0: 60.0
  p0*p1: -2.0
  n0*n1: 2.0
  p1*n1: 60.0


,p0,n0,p1,n1,s0,s1,E_qubo,E_cubica,Delta
0,0,1,0,1,-1,-1,-1.0,-1.0,0.0
1,0,1,0,0,-1,0,-1.0,-1.0,0.0
2,0,1,1,0,-1,1,1.0,1.0,0.0
3,0,0,0,1,0,-1,-2.0,-2.0,0.0
4,0,0,0,0,0,0,0.0,0.0,0.0
5,0,0,1,0,0,1,2.0,2.0,0.0
6,1,0,0,1,1,-1,-1.0,-1.0,0.0
7,1,0,0,0,1,0,1.0,1.0,0.0
8,1,0,1,0,1,1,1.0,1.0,0.0


Max |Delta|: 0.0
